# Working on these

- [ ] Benchmark from OpenAI x COVID-19-deepset with BERTScore
- [ ] Benchmark from petal.org
- [ ] Evaluate extractive and abstracting summary
- [ ] Mis-information safety mechanism
- [ ] Traceability (paragraphs)
- [ ] Traceability (words)

## Test set creation

- From `covid_qa_deepset`
- 8-2 split, rng = 2023, test n = 404

In [1]:
from askem.data import COVID_QA, GPTBench
from askem.openai import get_answer
from tqdm.notebook import tqdm

test_data = COVID_QA.train_test_split(test_size=0.2, seed=2023)["test"]
test_data

Found cached dataset covid_qa_deepset (/root/.cache/huggingface/datasets/covid_qa_deepset/covid_qa_deepset/1.0.0/fb886523842e312176f92ec8e01e77a08fa15a694f5741af6fc42796ee9c8c46)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached split indices for dataset at /root/.cache/huggingface/datasets/covid_qa_deepset/covid_qa_deepset/1.0.0/fb886523842e312176f92ec8e01e77a08fa15a694f5741af6fc42796ee9c8c46/cache-9cd84ba804afe701.arrow and /root/.cache/huggingface/datasets/covid_qa_deepset/covid_qa_deepset/1.0.0/fb886523842e312176f92ec8e01e77a08fa15a694f5741af6fc42796ee9c8c46/cache-9f5b527e317ef7fc.arrow


Dataset({
    features: ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'],
    num_rows: 404
})

## OpenAI

In [4]:
# Resume eval
done_ids = [row.id for row in GPTBench.select(GPTBench.id)]

for data in tqdm(test_data):
    if data["id"] in done_ids:
        continue

    gpt_answer = get_answer(data["context"], data["question"])
    GPTBench.insert(
        id=data["id"],
        context=data["context"],
        question=data["question"],
        true_answer=data["answers"],
        gpt_answer=gpt_answer,
    ).on_conflict_ignore().execute()

  0%|          | 0/404 [00:00<?, ?it/s]